In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train.csv")
test_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/test.csv")
solution_example=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv")
test_calenders_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv")
train_calenders_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv")

In [ ]:
train_data.dtypes,test_data.dtypes,train_data.shape

In [ ]:
train_columns=list(test_data.columns)
train_columns.append("orders")
train_data=train_data[train_columns]


In [ ]:
def add_additional_date_features(df):
    df["month"]=df.date.dt.month
    df["day_num_week"]=df.date.dt.weekday
    df["year"]=df.date.dt.year
    return df

In [ ]:
def find_outliers_iqr(df, column, threshold=1.5):
    """
    Identifies outliers in a DataFrame column using the IQR method.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column (str): The name of the column to check for outliers.
    threshold (float): The multiplication factor for the IQR.

    Returns:
    pd.DataFrame: A DataFrame containing the outliers.
    """

    # Calculate the IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (threshold * IQR)
    upper_bound = Q3 + (threshold * IQR)

    # Identify outliers
    #outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return lower_bound,upper_bound 

In [ ]:
def mark_outlier(df):
    lower_bound,upper_bound =find_outliers_iqr(df,"orders", threshold=1.5)
    df_outliers = df[(df["orders"] < lower_bound) | (df["orders"] > upper_bound)]
    df_non_outliers = df[(df["orders"] >= lower_bound) & (df["orders"] <=upper_bound)]
    df_outliers["IS_OUTLIER"]=1
    df_non_outliers["IS_OUTLIER"]=0
    df=pd.concat([df_outliers,df_non_outliers])
    return df

In [ ]:
def generate_dates(start_date, end_date):
    """generate all month start dates between two given dates(including the start and end date)
    Parameters:
              start_date(date) : starting date
              end_date(date) : ending date
    returns:
      a dataframe with column name "PERIOD" and values all month start dates between the start and end date

    """

    dates = []

    date=pd.to_datetime(start_date)
    while date<=pd.to_datetime(end_date):
       
        dates.append(date)
        date = date + timedelta(days=1)
        

    df = pd.DataFrame({"date": dates})
    df["date"] = pd.to_datetime(df["date"])
    return df


def add_missing_dates(df_p):
    "if any month start date is missing between the start and and end date identify those and add it to data frame"
    start_date = df_p.date.min()
    
    end_date = df_p.date.max()
    
    dates_df = generate_dates(start_date, end_date)
    df_p["date"]=pd.to_datetime(df_p["date"])

    df_filled = dates_df.merge(
        df_p,
        on=[
            "date"
        ],
        how="left",
    )
    df_filled["warehouse"]=df_filled["warehouse"].ffill()
    df_filled["id"]=df_filled["id"].ffill()
    df_filled["orders"]= df_filled["orders"].fillna(0)
    return df_filled
    

In [ ]:
def transform_data(df,warehouse):
    df["date"]=pd.to_datetime(df["date"])
    df=add_missing_dates(df)
    df= add_additional_date_features(df)
    encode_columns=['month','day_num_week','holiday_name']
    if warehouse=='Munich_1':
        transformed_train_data['orders']=(transformed_train_data['orders']
                                          .where(transformed_train_data['day_num_week']!=6,0))
    df=pd.get_dummies(df,columns=encode_columns)
    df=mark_outlier(df)
    return df
def transform_test_data(df,warehouse):
    df["date"]=pd.to_datetime(df["date"])
    df=add_missing_dates(df)
    df= add_additional_date_features(df)
    encode_columns=['month','day_num_week','holiday_name']
    df=pd.get_dummies(df,columns=encode_columns)
  
    return df 

In [ ]:
for warehouse in train_data.warehouse.unique():
    df=train_data[train_data.warehouse==warehouse]    
    transformed_train_data=transform_data(df,warehouse)   
    file_name=warehouse+"input_data.csv"
    transformed_train_data.to_csv(file_name)
    


In [ ]:
for warehouse in train_data.warehouse.unique():
    df=test_data[test_data.warehouse==warehouse]  
    transformed_test_data=transform_test_data(df,warehouse)
    file_name=warehouse+"test_data.csv"
    transformed_test_data.to_csv(file_name)